## PUBG.OP.GG match Details Scrape

Given a player_list

In [1]:
%load_ext autoreload
%autoreload 2

import requests
import pandas as pd
import datetime as dt
import numpy as np
import json
from helper import *

We start by importing previously generated player list

In [2]:
player_list = pd.read_csv('player_base(large).csv', index_col=0, header=None)


In [3]:
player_list = player_list.sample(player_list.shape[0] - 1)[1].values
player_list

array(['ZEUS2411', 'DMXPRO', 'MrBashis', ..., 'Nurtai', 'almightyloaf52',
       '16wu-li-lao-shi'], dtype=object)

We then pull each player's match history and save to a list of match IDs

In [4]:
def mp_get_scrape_match_list(player):
    player = str(player)
    try:
        uinfo = request_user_info(player)
    except:
        return []
    
    uid = get_user_id(uinfo)
    if uid is not None:
        try:
            return request_match_history(uid)
        except:
            return []
    
if __name__ == '__main__':
    results = mp_run(player_list, mp_get_scrape_match_list, 800)
    results = [i for i in results if i]

Scraping 726550 items using <function mp_get_scrape_match_list at 0x000001E2450ED9D0> in 9500 processes.
I HAVE COMPLETED!!!    


In [5]:
match_list = list(set(np.concatenate(list(results)))) # remove dupes after
print('%i match ids scraped' % len(match_list))

435408 match ids scraped


Separate Match IDs into chunks for multithreading process to generate separate files of 10000 match IDs

In [6]:
create_chunks = lambda l,n: [l[i:i + n] for i in range(0, len(l), n)]
chunk_size = 10000
chunks = create_chunks(match_list, chunk_size)
print('Saving %i chunks' % len(chunks))
np.save('match_id_chunks.npy', chunks, allow_pickle=True)

Saving 44 chunks


C:\Users\swrgo\anaconda3\lib\site-packages\numpy\core\_asarray.py:171: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order, subok=True)


In [7]:
chunks = np.load('match_id_chunks.npy',allow_pickle = True)
print(len(chunks))

44


### Match General Details


In [9]:
def mp_get_scrape_match_gen_details(m_id):
    d = []
    try:
        feed = request_match_detail(m_id)
        for team in feed['teams']:
            for player in team['participants']:
                d.append({
                    'match_id': m_id,
                    'date': feed['started_at'],
                    'match_mode': feed['mode'],
                    'party_size': feed['queue_size'],
                    'game_size': feed['total_rank'],
                    'team_id': team['_id'],
                    'team_placement': team['stats']['rank'],
                    'player_name': player['user']['nickname'],
                    'player_kills': player['stats']['combat']['kda']['kills'],
                    'player_assists': player['stats']['combat']['kda']['assists'],
                    'player_survive_time':  player['stats']['combat']['time_survived'],
                    'player_dmg': player['stats']['combat']['damage']['damage_dealt'],
                    'player_dbno': player['stats']['combat']['dbno']['knock_downs'],
                    'player_dist_walk': player['stats']['combat']['distance_traveled']['walk_distance'],
                    'player_dist_ride': player['stats']['combat']['distance_traveled']['ride_distance'],
                })
    except:
        return d
    return d

if __name__ == '__main__':
    for i,c in enumerate(chunks):
        results = mp_run(c, mp_get_scrape_match_gen_details, 800)
        match_gen_details = np.concatenate(list(results)).tolist()
        print('Saving %i rows of chunk %i to csv' % (len(match_gen_details), i))
        pd.DataFrame(match_gen_details).to_csv('gen_match_stats_%i.csv' % i, index=False)

Scraping 10000 items using <function mp_get_scrape_match_gen_details at 0x000001E24512C0D0> in 800 processes.
I HAVE COMPLETED!!!  
Saving 58301 rows of chunk 0 to csv
Scraping 10000 items using <function mp_get_scrape_match_gen_details at 0x000001E24512C0D0> in 800 processes.
I HAVE COMPLETED!!!  
Saving 94103 rows of chunk 1 to csv
Scraping 10000 items using <function mp_get_scrape_match_gen_details at 0x000001E24512C0D0> in 800 processes.
I HAVE COMPLETED!!!  
Saving 113446 rows of chunk 2 to csv
Scraping 10000 items using <function mp_get_scrape_match_gen_details at 0x000001E24512C0D0> in 800 processes.
I HAVE COMPLETED!!!  
Saving 136380 rows of chunk 3 to csv
Scraping 10000 items using <function mp_get_scrape_match_gen_details at 0x000001E24512C0D0> in 800 processes.
I HAVE COMPLETED!!!  
Saving 172343 rows of chunk 4 to csv
Scraping 10000 items using <function mp_get_scrape_match_gen_details at 0x000001E24512C0D0> in 800 processes.
I HAVE COMPLETED!!!  
Saving 215476 rows of chu

### Match Kills Details

In [11]:
def mp_get_scrape_match_kill_details(m_id):
    d = []
    try:
        feed = request_match_kill_detail(m_id)
        for death in feed['deaths']:
            if death['killer'] is None:
                killer_name = None; killer_placement = None; killer_position_x = None; killer_position_y = None;
            else:
                killer_name = death['killer']['user']['nickname']
                killer_placement = death['killer']['rank'] if death['killer']['rank'] is not None else 1
                killer_position_x = death['killer']['position']['x']
                killer_position_y = death['killer']['position']['y']

            d.append({
                'match_id': m_id,
                'time': death['time_event'],
                'map': feed['map_id'],
                'killed_by': death['description'],
                'killer_name': killer_name,
                'killer_placement': killer_placement,
                'killer_position_x': killer_position_x,
                'killer_position_y': killer_position_y,
                'victim_name': death['victim']['user']['nickname'],
                'victim_placement': death['victim']['rank'],
                'victim_position_x': death['victim']['position']['x'],
                'victim_position_y': death['victim']['position']['y'],
            })
    except:
        return d
    return d

if __name__ == '__main__':
    for i,c in enumerate(chunks):
        results = mp_run(c, mp_get_scrape_match_kill_details, 800)
        match_kill_details = np.concatenate(list(results)).tolist()
        print('Saving %i rows of chunk %i to csv' % (len(match_kill_details), i))
        pd.DataFrame(match_kill_details).to_csv('match_kill_match_stats_%i.csv' % i, index=False)

Scraping 10000 items using <function mp_get_scrape_match_kill_details at 0x000001E249F4EB80> in 800 processes.
I HAVE COMPLETED!!!  
Saving 51767 rows of chunk 0 to csv
Scraping 10000 items using <function mp_get_scrape_match_kill_details at 0x000001E249F4EB80> in 800 processes.
I HAVE COMPLETED!!!  
Saving 53933 rows of chunk 1 to csv
Scraping 10000 items using <function mp_get_scrape_match_kill_details at 0x000001E249F4EB80> in 800 processes.
I HAVE COMPLETED!!!  
Saving 57508 rows of chunk 2 to csv
Scraping 10000 items using <function mp_get_scrape_match_kill_details at 0x000001E249F4EB80> in 800 processes.
I HAVE COMPLETED!!!  
Saving 67898 rows of chunk 3 to csv
Scraping 10000 items using <function mp_get_scrape_match_kill_details at 0x000001E249F4EB80> in 800 processes.
I HAVE COMPLETED!!!  
Saving 70885 rows of chunk 4 to csv
Scraping 10000 items using <function mp_get_scrape_match_kill_details at 0x000001E249F4EB80> in 800 processes.
I HAVE COMPLETED!!!  
Saving 52490 rows of c